In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import asyncio
import scipy.io as sio
from scipy.io import loadmat  # Import loadmat
# import matlab.engine
from matplotlib.colors import ListedColormap
import time
import os
import multiprocessing as mp
from scipy.interpolate import griddata
from scipy.interpolate import interp2d, RegularGridInterpolator
from scipy.linalg import cholesky
import sampyl as smp
import sys
import multiprocessing as mp
from numba import jit
import csv
from matplotlib.colors import ListedColormap
# Set print options to increase precision
np.set_printoptions(precision=20, suppress=False, formatter={'complexfloat': '{: 0.4f}'.format})
from concurrent.futures import ProcessPoolExecutor
from scipy.io import savemat
import gstools as gs
from gstools import SRF, Gaussian
from gstools.random import MasterRNG
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap,LinearSegmentedColormap

In [ ]:
def creating_array_3bingham_distributions(num):
    # structured field with a size of 100x100 and a grid-size of 1x1
    x = y = range(26)

    model = Gaussian(dim=2, var=1, len_scale=500)
    srf = SRF(model)
    grf_array = srf.structured([x, y])
    # srf1.plot()
    # print(grf_array.shape)a
    # plt.imshow(grf_array)
    # plt.colorbar()  # To show the color scale
    # plt.savefig("data/grfplot_{}.png".format(num))
    # plt.close()
    # plt.show()

    # Dynamically generate random values for percentiles
    percentile1 = np.random.uniform(20, 50)  # Random percentile between 20 and 50
    percentile2 = np.random.uniform(51, 90)  # Random percentile between 51 and 90
    minimum1 = grf_array.min()
    maximum1 = grf_array.max()

    # Calculate the thresholds based on the dynamically chosen percentiles
    threshold1 = np.percentile(grf_array, percentile1)
    threshold2 = np.percentile(grf_array, percentile2)

    # print(f"Minimum: {minimum1}")
    # print(f"Maximum: {maximum1}")
    print(f"30th Percentile (Dynamic): {percentile1}")
    print(f"50th Percentile (Dynamic): {percentile2}")

    resultant_array_3distributions = np.zeros((26,26))

    for i in range(26):
        for j in range(26):
            if grf_array[i][j] <= threshold1:
                resultant_array_3distributions[i][j] = 0
            elif threshold1 < grf_array[i][j] <= threshold2:
                resultant_array_3distributions[i][j] = 0.5
            else:
                resultant_array_3distributions[i][j] = 1

    # Create a custom color map for the final visualization
    cmap = LinearSegmentedColormap.from_list("custom_cmap", ["black", "purple", "yellow"])

    # Plot the resulting array
    plt.imshow(resultant_array_3distributions, cmap=cmap)
    plt.colorbar()  # To show the color scale
    plt.savefig("data/result_{}.png".format(num))
    plt.close()
    # plt.show()
    return resultant_array_3distributions

In [ ]:
def creating_array_2bingham_distributions(num):
    # structured field with a size of 100x100 and a grid-size of 1x1
    x = y = range(26)

    model = Gaussian(dim=2, var=1, len_scale=500)
    srf = SRF(model)
    grf_array = srf.structured([x, y])
    # srf1.plot()
    # plt.imshow(grf_array)
    # plt.colorbar()  # To show the color scale
    # plt.savefig("data/grfplot_{}.png".format(num))
    # plt.show()

    # Dynamically calculate thresholds based on percentiles
    percentile = np.random.uniform(20, 80)
    threshold = np.percentile(grf_array, percentile)
    minimum1 = grf_array.min()
    maximum1 = grf_array.max()

    # print(f"Minimum: {minimum1}")
    # print(f"Maximum: {maximum1}")
    print(f"30th Percentile (Dynamic): {percentile}")

    resultant_array_2distributions = np.zeros((26,26))

    for i in range(26):
        for j in range(26):
            if grf_array[i][j] <= threshold:
                resultant_array_2distributions[i][j] = 0
            else:
                resultant_array_2distributions[i][j] = 1

    # Create a custom color map for the final visualization
    cmap = LinearSegmentedColormap.from_list("custom_cmap", ["black", "yellow"])

    # Plot the resulting array
    plt.imshow(resultant_array_2distributions, cmap=cmap)
    plt.colorbar()  # To show the color scale
    plt.savefig("data/result_{}.png".format(num))
    # plt.show()
    return resultant_array_2distributions

In [ ]:
def background(internal_loop):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, internal_loop, *args, **kwargs)
    return wrapped

class AII_model:
    def __init__(self, array, eta1, kappa1, mu_phi1, mu_theta1, eta2, kappa2, mu_phi2, mu_theta2, eta3, kappa3, mu_phi3, mu_theta3):
        # eta, kappa = 0.25 , 0.05
        self.condRef = 8.7e5
        self.sigxx = 8.97e5
        self.sigzz = 8.43e5
        self.coil_rad_x = 2.1509e6
        self.coil_rad_y = 2.1509e6
        self.eta1 = eta1
        self.kappa1 = kappa1
        self.mu_phi1 = mu_phi1
        self.mu_theta1 = mu_theta1
        self.eta2 = eta2
        self.kappa2 = kappa2
        self.mu_phi2 = mu_phi2
        self.mu_theta2 = mu_theta2
        self.eta3 = eta3
        self.kappa3 = kappa3
        self.mu_phi3 = mu_phi3
        self.mu_theta3 = mu_theta3
        self.data_to_append = []
        self.phi_theta_sigma_data = {}
        self.array = array
        self.V1 , self.V2, self.V3 , self.lambda1, self.lambda2, self.lambda3, \
        self.V21 , self.V22, self.V23 , self.lambda21, self.lambda22, self.lambda23, \
        self.V31 , self.V32, self.V33, self.lambda31, self.lambda32, self.lambda33 = self.compute_bingham_distributions()
        self.pdf_value = self.calculate_mx_value()
        self.candidates = []
        self.eulerangles = []
        self.microstructure_list = {}
        self.number_of_microstructures = 1

    async def async_init(self):
        await self.run_simulation()

    def compute_bingham_distributions(self):
        V1 = np.array([
            np.cos(self.mu_theta1/2)*np.cos(self.mu_phi1/2),
            -1*np.sin(self.mu_theta1/2)*np.cos(self.mu_phi1/2),
            -1*np.sin(self.mu_theta1/2)*np.sin(self.mu_phi1/2),
            -1*np.cos(self.mu_theta1/2)*np.sin(self.mu_phi1/2)
        ])
        V2 = np.array([
            -0.5 * np.cos(self.mu_theta1/2) * np.sin(self.mu_phi1 / 2),
            0.5 * np.sin(self.mu_theta1/2) * np.sin(self.mu_phi1 / 2),
            -0.5 * np.sin(self.mu_theta1/2) * np.cos(self.mu_phi1 / 2),
            -0.5 * np.cos(self.mu_theta1/2) * np.cos(self.mu_phi1 / 2)
        ])
        V3 = np.array([
            -0.5 * np.sin(self.mu_theta1/2) * np.cos(self.mu_phi1 / 2),
            -0.5 * np.cos(self.mu_theta1/2) * np.cos(self.mu_phi1 / 2),
            -0.5 * np.cos(self.mu_theta1/2) * np.sin(self.mu_phi1 / 2),
            0.5 * np.sin(self.mu_theta1/2) * np.sin(self.mu_phi1 / 2)
        ])
        l1 = -0.5
        l2 = l1 / self.eta1
        l3 = l2 / self.kappa1
        V21 = np.array([
            np.cos(self.mu_theta2/2)*np.cos(self.mu_phi2/2),
            -1*np.sin(self.mu_theta2/2)*np.cos(self.mu_phi2/2),
            -1*np.sin(self.mu_theta2/2)*np.sin(self.mu_phi2/2),
            -1*np.cos(self.mu_theta2/2)*np.sin(self.mu_phi2/2)
        ])
        V22 = np.array([
            -0.5 * np.cos(self.mu_theta2/2) * np.sin(self.mu_phi2 / 2),
            0.5 * np.sin(self.mu_theta2/2) * np.sin(self.mu_phi2 / 2),
            -0.5 * np.sin(self.mu_theta2/2) * np.cos(self.mu_phi2 / 2),
            -0.5 * np.cos(self.mu_theta2/2) * np.cos(self.mu_phi2 / 2)
        ])
        V23 = np.array([
            -0.5 * np.sin(self.mu_theta2/2) * np.cos(self.mu_phi2 / 2),
            -0.5 * np.cos(self.mu_theta2/2) * np.cos(self.mu_phi2 / 2),
            -0.5 * np.cos(self.mu_theta2/2) * np.sin(self.mu_phi2 / 2),
            0.5 * np.sin(self.mu_theta2/2) * np.sin(self.mu_phi2 / 2)
        ])
        l21 = -0.5
        l22 = l21 / self.eta2
        l23 = l22 / self.kappa2
        V31 = np.array([
            np.cos(self.mu_theta3/2)*np.cos(self.mu_phi3/2),
            -1*np.sin(self.mu_theta3/2)*np.cos(self.mu_phi3/2),
            -1*np.sin(self.mu_theta3/2)*np.sin(self.mu_phi3/2),
            -1*np.cos(self.mu_theta3/2)*np.sin(self.mu_phi3/2)
        ])
        V32 = np.array([
            -0.5 * np.cos(self.mu_theta3/2) * np.sin(self.mu_phi3 / 2),
            0.5 * np.sin(self.mu_theta3/2) * np.sin(self.mu_phi3 / 2),
            -0.5 * np.sin(self.mu_theta3/2) * np.cos(self.mu_phi3 / 2),
            -0.5 * np.cos(self.mu_theta3/2) * np.cos(self.mu_phi3 / 2)
        ])
        V33 = np.array([
            -0.5 * np.sin(self.mu_theta3/2) * np.cos(self.mu_phi3 / 2),
            -0.5 * np.cos(self.mu_theta3/2) * np.cos(self.mu_phi3 / 2),
            -0.5 * np.cos(self.mu_theta3/2) * np.sin(self.mu_phi3 / 2),
            0.5 * np.sin(self.mu_theta3/2) * np.sin(self.mu_phi3 / 2)
        ])
        l31 = -0.5
        l32 = l1 / self.eta3
        l33 = l2 / self.kappa3
        
        return V1 , V2 , V3 , l1 , l2 , l3 , V21 , V22 , V23 , l21 , l22 , l23 , V31 , V32 , V33 , l31 , l32 , l33 

    def pi1_x(self, x):
        term1 = self.lambda1 * (np.dot(self.V1.T, x)) ** 2
        term2 = self.lambda2 * (np.dot(self.V2.T, x)) ** 2
        term3 = self.lambda3 * (np.dot(self.V3.T, x)) ** 2

        sum_terms = term1 + term2 + term3

        pi_x_value = np.exp(sum_terms)
        return pi_x_value
    
    def pi2_x(self, x):
        term1 = self.lambda21 * (np.dot(self.V21.T, x)) ** 2
        term2 = self.lambda22 * (np.dot(self.V22.T, x)) ** 2
        term3 = self.lambda23 * (np.dot(self.V23.T, x)) ** 2

        sum_terms = term1 + term2 + term3

        pi_x_value = np.exp(sum_terms)
        return pi_x_value
    
    def pi3_x(self, x):
        term1 = self.lambda31 * (np.dot(self.V31.T, x)) ** 2
        term2 = self.lambda32 * (np.dot(self.V32.T, x)) ** 2
        term3 = self.lambda33 * (np.dot(self.V33.T, x)) ** 2

        sum_terms = term1 + term2 + term3

        pi_x_value = np.exp(sum_terms)
        return pi_x_value
    
    def calculate_mx_value(self):
        return 1
            
    def rununtiltrue(self, number):
        while True:
            x_candidate = self.generaterandomquaternion()
            if number==0:
                f_candidate = self.pi1_x(x_candidate)
            elif number==0.5:
                f_candidate = self.pi2_x(x_candidate)
            elif number==1:
                f_candidate = self.pi3_x(x_candidate)
            alpha = f_candidate / self.pdf_value
            if alpha >= 0.98:
                return x_candidate

    def generaterandomquaternion(self):
        # Sample three uniform random variables
        u1, u2, u3 = np.random.rand(3)

        # Compute the quaternion components
        q = np.array([
            np.sqrt(1-u1) * np.sin(2*np.pi*u2),
            np.sqrt(1-u1) * np.cos(2*np.pi*u2),
            np.sqrt(u1) * np.sin(2*np.pi*u3),
            np.sqrt(u1) * np.cos(2*np.pi*u3)
        ])
        return q
    
    @staticmethod
    def quaternion_to_rotation_matrix(q):
        # Compute the associated rotation matrix from the quaternion
        R = np.array([
            [np.square(q[0]) + np.square(q[1]) - np.square(q[2]) - np.square(q[3]), 2*(q[1]*q[2] + q[0]*q[3]), 2*(q[1]*q[3] - q[0]*q[2])],
            [2*(q[1]*q[2] - q[0]*q[3]), np.square(q[0]) - np.square(q[1]) + np.square(q[2]) - np.square(q[3]), 2*(q[3]*q[2] + q[0]*q[1])],
            [2*(q[3]*q[1] + q[0]*q[2]), 2*(q[3]*q[2] - q[0]*q[1]), np.square(q[0]) - np.square(q[1]) - np.square(q[2]) + np.square(q[3])]
        ])
        return R
    
    @staticmethod
    def rotation_matrix_to_euler_angles(R):
        second_angle = np.arccos(R[2][2])
        if second_angle > 0:
            first_angle = np.arctan2(R[2][0], -R[2][1])
            third_angle = np.arctan2(R[0][2], R[1][2])
        elif second_angle == 0:
            first_angle = np.arctan2(R[0][1], R[0][0])
            third_angle = 0
        return first_angle, second_angle, third_angle

    @staticmethod
    def quaternion_angular_difference(q1, q2):
        # Ensure the quaternions are normalized
        q1 = q1 / np.linalg.norm(q1)
        q2 = q2 / np.linalg.norm(q2)
        # Calculate the dot product between the two quaternions
        dot_product = np.dot(q1, q2)
        # Clamp the dot product to avoid numerical issues
        dot_product = np.clip(dot_product, -1.0, 1.0)
        # Calculate the angular difference in radians
        angle = 2 * np.arccos(dot_product)
        # Convert radians to degrees
        return angle
    
    async def process_element(self, i, j, number):
        # x_candidate = []
        if number==0:
            x_candidate = self.rununtiltrue(number)
        elif number==0.5:
            x_candidate = self.rununtiltrue(number)
        elif number==1:
            x_candidate = self.rununtiltrue(number)
        self.candidates.append(x_candidate)
        angle = self.rotation_matrix_to_euler_angles(self.quaternion_to_rotation_matrix(x_candidate))
        first, second, third = angle
        self.eulerangles.append(angle)
        self.data_to_append.append((i, j, first, second, third))
            
    async def process_microstructure(self, val):
        """Process each microstructure in parallel."""
        tasks = []
        for i in range(26):
            for j in range(26):
                if self.array[i][j]==0:
                    tasks.append(self.process_element(i, j, 0))
                elif self.array[i][j]==0.5:
                    tasks.append(self.process_element(i, j, 0.5))
                elif self.array[i][j]==1:
                    tasks.append(self.process_element(i, j, 1))
        await asyncio.gather(*tasks)
        self.data_to_append.sort(key=lambda x: (x[0], x[1]))
        self.microstructure_list[val] = self.data_to_append
#         print(self.data_to_append)
        # print(f"Printing for {val} microstructure")
    
    async def run_simulation(self):
        """Parallelize the outer loop using asyncio tasks."""
        start_time = time.time()
        outer_tasks = [self.process_microstructure(val) for val in range(1, self.number_of_microstructures+1)]
        await asyncio.gather(*outer_tasks)
        end_time = time.time() - start_time
        print(f"Time taken by run_simulation: {end_time} seconds")

    def write_data_to_new_file(self,file_path):
        # Check if the file exists, delete it if it does
        if os.path.exists(file_path):
            os.remove(file_path)
        # Open the file in append mode ('a') or create it if it doesn't exist ('a+')
        with open(file_path, 'a') as file:
            # Append each line of data to the file
            for line in self.data_to_append[:]:
                file.write(' '.join(map(str, line)) + '\n')  # Add a newline character to separate entries
                    
    def write_data_to_new_file2(self, file_path):
        # Check if the file exists, delete it if it does
        if os.path.exists(file_path):
            os.remove(file_path)

        # Open the file in write mode ('w') to create or overwrite the file
        with open(file_path, 'w') as file:
            # Reshape the 1x676 data into 26x26 format and write each row to a new line
            for i in range(26):
                row_data = self.data_to_append[i*26:(i+1)*26]  # Get the data for the current row
                file.write(' '.join(map(str, row_data)) + '\n')  # Write the row to the file

    def scan_operation(self): 
        microstructure = self.microstructure_list[1]
        # Extracting X, Y, phi, and theta values from each microstructure
        X = np.array([tup[0] for tup in microstructure])      # * (6176/25) + 8
        X = X.reshape((26, 26)).T
        X = np.stack((X, X), axis=2)

        Y = np.array([tup[1] for tup in microstructure])      #* (6312/25) + 8
        Y = Y.reshape((26, 26)).T
        Y = np.stack((Y, Y), axis=2)

        self.X = X
        self.Y = Y
            
        for val, microstructure in self.microstructure_list.items():

            phi = np.array([tup[2] for tup in microstructure])
            phi = phi.reshape((26, 26)).T
            phi = np.stack((phi, phi), axis=2)
            cphi = np.cos(phi)
            sphi = np.sin(phi)

            theta = np.array([tup[3] for tup in microstructure])
            theta = theta.reshape((26, 26)).T
            theta = np.stack((theta, theta), axis=2)
            stheta = np.sin(theta)

            # Optionally, you can store these processed arrays in a dictionary if needed
            self.microstructure_list[val] = {
                'cphi': cphi,
                'sphi': sphi,
                'stheta': stheta,
                'microstructure': microstructure
            }
        
        scanDims = {'scanStart': np.array([0, 0, 0]), 'scanLength': np.array([5, 5, 5]), 'scanStep': np.array([0.2, 0.2, 0.2])}

        #scanDims['scanStart'] *= 1000
        #scanDims['scanLength'] *= 1000
        #scanDims['scanStep'] *= 1000

        # Calculate number of points in each dimension
        numXPoints = int(scanDims['scanLength'][0] / scanDims['scanStep'][0]) + 1
        numYPoints = int(scanDims['scanLength'][1] / scanDims['scanStep'][1]) + 1

        scanDims['scanStep'] = [1,1,1];
        
        # Set up the scan structure
        scan = np.zeros((numXPoints, numYPoints), dtype=object)
        for ixpos in range(numXPoints):
            for iypos in range(numYPoints):
                scan[iypos, ixpos] = {
                    'num': 0,
                    'pos': [
                        scanDims['scanStart'][0] + scanDims['scanStep'][0] * ixpos,
                        scanDims['scanStart'][1] + scanDims['scanStep'][1] * iypos
                    ],
                    'area': [],
                    'total': 0
                }

        Exs = np.load('Exs_26_26_updated_iterations.npy')
        Eys = np.load('Eys_26_26_updated_iterations.npy')

        mul = np.zeros(scan.shape, dtype=np.complex128)

        for val, microstructure_data in self.microstructure_list.items():
            # Initialize scanPoint
            scanPoint = 0

            # Initialize ectScan with the same structure as scan but with all elements set to zero
            ectScan = {
                'val': val,
                'XD': np.zeros(scan.shape, dtype=float),
                'YD': np.zeros(scan.shape, dtype=float),
                'zl': np.zeros(scan.shape, dtype=np.complex128),
                'Xsub_slice': np.empty(scan.shape, dtype=object),  
                'Ysub_slice': np.empty(scan.shape, dtype=object),  
                'Zsub_slice': np.empty(scan.shape, dtype=object)
            }

            sphi_sub = microstructure_data['sphi']
            stheta_sub = microstructure_data['stheta']
            cphi_sub = microstructure_data['cphi']

            # Compute sigma values once per microstructure
            sigma_11 = self.sigxx + (self.sigzz - self.sigxx) * sphi_sub ** 2 * stheta_sub ** 2
            sigma_12 = ((self.sigxx - self.sigzz) * (sphi_sub * cphi_sub * (stheta_sub ** 2))) / 2
            sigma_22 = self.sigxx + ((self.sigzz - self.sigxx) * (stheta_sub ** 2)) + ((self.sigxx - self.sigzz) * (sphi_sub ** 2) * (stheta_sub ** 2))

            numX, numY = scan.shape
            real_partf_list = np.empty(scan.shape, dtype=object)
            imag_partf_list = np.empty(scan.shape, dtype=object)
            for ix in range(numX):
                for iy in range(numY):

                    X = self.X
                    Y = self.Y
                    x = np.squeeze(X[0, :, 0])
                    y = np.squeeze(Y[:, 0, 0])

                    # Min and max distances from coil to position in x and y directions
                    #cx_lim_low = scan[ix, iy]['pos'][0] - self.coil_rad_x
                    #cx_lim_hi = scan[ix, iy]['pos'][0] + self.coil_rad_x
                    #cy_lim_low = scan[ix, iy]['pos'][1] - self.coil_rad_y
                    #cy_lim_hi = scan[ix, iy]['pos'][1] + self.coil_rad_y

                    # Find the indices for x and y using numpy
                    #lowIndX = np.argmax(x >= cx_lim_low)
                    #highIndX = np.argmax(x >= cx_lim_hi) - 1 if np.any(x >= cx_lim_hi) else len(x) - 1
                    #lowIndY = np.argmax(y >= cy_lim_low)
                    #highIndY = np.argmax(y >= cy_lim_hi) - 1 if np.any(y >= cy_lim_hi) else len(y) - 1
                    #lowIndZ = 0
                    #highIndZ = 1

                    # Extract subarrays for X, Y, Z

                    #Xsub = X[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][0]

                    #Ysub = Y[lowIndY:highIndY+1, lowIndX:highIndX+1, lowIndZ:highIndZ+1] - scan[ix, iy]['pos'][1]

                    Xsub = X - scan[ix, iy]['pos'][0]
                    Ysub = Y - scan[ix, iy]['pos'][1]
                    array1 = np.ones((X.shape[0], X.shape[1], 1)) * 8
                    array2 = np.ones((X.shape[0], X.shape[1], 1)) * 72

                    # Stack these arrays along the third dimension
                    Zsub = np.concatenate((array1, array2), axis=2)
                    # Ex = griddata(points, vals_x, (Xsub, Ysub, Zsub) , method='nearest')
                    # Ey = griddata(points, vals_y, (Xsub, Ysub, Zsub) , method='nearest')

                    Ex = Exs[scanPoint]   
                    Ey = Eys[scanPoint]

                    # Components of the approximate impedance integral (AII)
                    f1 = Ex ** 2 * (self.condRef - sigma_11)
                    f2 = -2 * Ex * Ey * sigma_12
                    f3 = Ey ** 2 * (self.condRef - sigma_22)
                    f = f1 + f2 + f3
                    real_partf_list[iy, ix] = np.real(f)
                    imag_partf_list[iy, ix] = np.imag(f)
                    
                    Xsub_slice = (np.squeeze(Xsub[0, :, 0]) / 1e6)
                    Ysub_slice = (np.squeeze(Ysub[:, 0, 0]) / 1e6)
                    Zsub_slice = (np.squeeze(Zsub[0, 0, :]) / 1e6)
                    
#                     Xsub_slice = np.squeeze(Xsub[0, :, 0])
#                     Ysub_slice = np.squeeze(Ysub[:, 0, 0])
#                     Zsub_slice = np.squeeze(Zsub[0, 0, :])
                                  
                    integral_inner = np.trapz(f, x=Xsub_slice, axis=1).reshape(26,1,2)
                    integral_middle = np.trapz(integral_inner, x=Ysub_slice, axis=0).reshape(1,1,2)
                    zl = np.trapz(integral_middle, x=Zsub_slice, axis=2)

                    #ectScan['XD'][ix, iy] = scan[ix, iy]['pos'][0] / 1000  # Back to mm
                    #ectScan['YD'][ix, iy] = scan[ix, iy]['pos'][1] / 1000  # Back to mm
                    ectScan['Xsub_slice'][iy, ix] = Xsub_slice
                    ectScan['Ysub_slice'][iy, ix] = Ysub_slice
                    ectScan['Zsub_slice'][iy, ix] = Zsub_slice
                    ectScan['zl'][iy, ix] = zl
                    
                    mul[iy, ix] += ectScan['zl'][iy, ix]
                    
                    scanPoint += 1
            microstructure_data['ectScan'] = ectScan
            microstructure_data['realpartlist'] = real_partf_list
            microstructure_data['imagpartlist'] = imag_partf_list

        mul = mul/self.number_of_microstructures

        # Store the 'mul' result back in self if needed
        self.mul = mul
        return ectScan

    def calculate_covariance_matrix(self):
        num_microstructures = len(self.microstructure_list)
        num_elements = 676  # 26 * 26
        single_side = 26
        # Initialize covariance matrices
        gamma = np.zeros((num_elements, num_elements), dtype=np.complex128)
        gamma_hat = np.zeros((num_elements, num_elements), dtype=float)
        
        for l in range(num_elements):
            for k in range(l + 1):
                sum_zl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    ectScan = microstructure_data['ectScan']
                    zl_flatten = ectScan['zl'].flatten()
                    # Get zl for l and k
                    zl_l = zl_flatten[l]
                    zl_k = zl_flatten[k]

                    sum_zl_product += zl_l * zl_k 
                    
                gamma[l, k] = sum_zl_product / self.number_of_microstructures  - (self.mul.flat[l]*self.mul.flat[k])
                gamma[k, l] = gamma[l, k]
            
        self.gamma = gamma
        
        for l in range(num_elements):  # num_elements is the size of the matrix
            for k in range(l + 1):
                sum_zlk_product = 0
                sum_zkl_product = 0
                for val, microstructure_data in self.microstructure_list.items():
                    i, j = divmod(l, single_side)
                    m, n = divmod(k, single_side)

                    realpartl = microstructure_data['realpartlist'][i,j]
                    imagpartl = microstructure_data['imagpartlist'][i,j]
                    
                    realpartk = microstructure_data['realpartlist'][m,n]
                    imagpartk = microstructure_data['imagpartlist'][m,n]
                    
                    Xsub_slicel = microstructure_data['ectScan']['Xsub_slice'][i,j]
                    Ysub_slicel = microstructure_data['ectScan']['Ysub_slice'][i,j]
                    Zsub_slicel = microstructure_data['ectScan']['Zsub_slice'][i,j]
                    
                    Xsub_slicek = microstructure_data['ectScan']['Xsub_slice'][m,n]
                    Ysub_slicek = microstructure_data['ectScan']['Ysub_slice'][m,n]
                    Zsub_slicek = microstructure_data['ectScan']['Zsub_slice'][m,n]
                    
                    integral_inner_l_lk = np.trapz(realpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_lk = np.trapz(integral_inner_l_lk, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zlk_l = np.trapz(integral_middle_l_lk, x=Zsub_slicel, axis=2)

                    integral_inner_k_lk = np.trapz(imagpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_lk = np.trapz(integral_inner_k_lk, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zlk_k = np.trapz(integral_middle_k_lk, x=Zsub_slicek, axis=2)

                    integral_inner_k_kl = np.trapz(realpartk, x=Xsub_slicek, axis=1).reshape(25,1,2)
                    integral_middle_k_kl = np.trapz(integral_inner_k_kl, x=Ysub_slicek, axis=0).reshape(1,1,2)
                    zkl_k = np.trapz(integral_middle_k_kl, x=Zsub_slicek, axis=2)

                    integral_inner_l_kl = np.trapz(imagpartl, x=Xsub_slicel, axis=1).reshape(25,1,2)
                    integral_middle_l_kl = np.trapz(integral_inner_l_kl, x=Ysub_slicel, axis=0).reshape(1,1,2)
                    zkl_l = np.trapz(integral_middle_l_kl, x=Zsub_slicel, axis=2)
                    
                    # Compute product and accumulate
                    sum_zlk_product += zlk_l * zlk_k
                    sum_zkl_product += zkl_l * zkl_k

                # Calculate the average across all microstructures
                gamma_hat[l, k] = sum_zlk_product / self.number_of_microstructures  - (np.real(self.mul.flat[l])*np.imag(self.mul.flat[k]))
                gamma_hat[k, l] = sum_zkl_product / self.number_of_microstructures  - (np.imag(self.mul.flat[l])*np.real(self.mul.flat[k]))

        self.gamma_hat = gamma_hat

In [ ]:
# Some instances are generated by creating_array_2bingham_distributions function and some are generated by creating_array_3bingham_distributions function.
# In the main code, I am using three numbers, 0, 0.5 and 1 and generating bingham distributions based on that.
# (26, 26, 3) for input is basically three euler angles for 26*26 different pixels.
# (26, 26, 3) for output is basically the real part of sctscan, the imaginary part of ectscan and 1.

# Lists to store all input and output data
# Create CSV writers for input and output .npy file paths
with open('data/input_data_paths.csv', mode='w', newline='') as input_csv_file, \
     open('data/output_data_paths.csv', mode='w', newline='') as output_csv_file:

    # Create CSV writers
    input_writer = csv.writer(input_csv_file)
    output_writer = csv.writer(output_csv_file)

    # Write headers (optional)
    input_writer.writerow(['input_data_path'])
    output_writer.writerow(['output_data_path'])

    input_data_2bingham_distributions = []
    output_data_2bingham_distributions = []
    for i in range(1,501):
        data = creating_array_2bingham_distributions(i)
        # Dynamically generate the parameters for AII_model
        eta1 = np.random.uniform(0.18, 0.25)  # Varies from 0.15 to 0.26
        kappa1 = np.random.uniform(0.02, 0.1)  # Varies from 0.02 to 0.1
        mu_phi1 = np.random.uniform(0, np.pi)  # Varies from 0 to pi
        mu_theta1 = np.random.uniform(0, np.pi/2)  # Varies from 0 to pi/2

        eta2 = np.random.uniform(0.18, 0.25)  # Varies from 0.15 to 0.26
        kappa2 = np.random.uniform(0.02, 0.1)  # Varies from 0.02 to 0.1
        mu_phi2 = np.random.uniform(0, np.pi)  # Varies from 0 to pi
        mu_theta2 = np.random.uniform(0, np.pi/2)  # Varies from 0 to pi/2

        eta3 = np.random.uniform(0.18, 0.25)  # Varies from 0.15 to 0.26
        kappa3 = np.random.uniform(0.02, 0.1)  # Varies from 0.02 to 0.1
        mu_phi3 = np.random.uniform(0, np.pi)  # Varies from 0 to pi
        mu_theta3 = np.random.uniform(0, np.pi/2)  # Varies from 0 to pi/2

        aiimodel = AII_model(data, eta1, kappa1, mu_phi1, mu_theta1, eta2, kappa2, mu_phi2, mu_theta2, eta3, kappa3, mu_phi3, mu_theta3)
        await aiimodel.async_init()
        inputdataarray = np.array([entry[2:] for entry in aiimodel.data_to_append])
        inputdataarray = inputdataarray.reshape(26, 26, 3)
        # aiimodel.write_data_to_new_file('data2.txt')
        ectScan = aiimodel.scan_operation()
        plt.close()
        plt.imshow(np.real(ectScan['zl']))
        plt.colorbar()
        plt.savefig("data/ectscan_real_{}.png".format(i))
        plt.close()
        plt.imshow(np.imag(ectScan['zl']))
        plt.colorbar()
        plt.savefig("data/ectscan_imag_{}.png".format(i))
        plt.close()
        outputdataarray = np.array([(np.real(ectScan['zl'][i, j]), np.imag(ectScan['zl'][i, j]), 1) for i in range(26) for j in range(26)])
        outputdataarray = outputdataarray.reshape(26, 26, 3)
        # Save inputdataarray and outputdataarray as .npy files
        input_file_path = f"data/inputdataarray_{i}.npy"
        output_file_path = f"data/outputdataarray_{i}.npy"
        txt_file_path = f"data/data_{i}.txt"
        aiimodel.write_data_to_new_file(txt_file_path)
        np.save(input_file_path, inputdataarray)
        np.save(output_file_path, outputdataarray)

        # Append input and output data to the lists
        input_data_2bingham_distributions.append(inputdataarray)
        output_data_2bingham_distributions.append(outputdataarray)
        
        # Write file paths to the CSV files
        input_writer.writerow([input_file_path])
        output_writer.writerow([output_file_path])